# Assign NERC labels to plants using 860 data and k-nearest neighbors

## Instructions
Make sure the `file_date` parameter below is set to whatever value you would like appended to file names.

Change the `most_recent_860_year` parameter below to match the most up-to-date EIA-860 annual data file. As of March 2018 this is 2016.

EIA-860 (annual) excel files will need to be [downloaded](https://www.eia.gov/electricity/data/eia860/) and unzipped to the `EIA downloads` folder. Make sure that all years from 2012 through the most recent data year are available. Also download the most recent [EIA-860m](https://www.eia.gov/electricity/data/eia860m/) to `EIA downloads`.

The most recent annual 860 file available (as of March 2018) represents 2016 data. When newer EIA-860 annual files are added the dictionary with pandas `read_excel` parameters will need to be updated. Note that EIA puts out an Early Release version of 860 with extra header rows and columns, so be sure to appropriately adjust the `skiprows` and `usecols` parameters if using an Early Release file.

The entire notebook can be run at once using *Run All Cells*

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
from os.path import join
import pandas as pd
from sklearn import neighbors, metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from collections import Counter
from copy import deepcopy


cwd = os.getcwd()
data_path = join(cwd, '..', 'Data storage')

### Date string for filenames
This will be inserted into all filenames (reading and writing)

In [2]:
file_date = '2018-03-06'
most_recent_860_year = 2016

## Load data
This loads facility data that has been assembled from the EIA bulk data file, and EIA-860 excel files. The EIA-860 excel files need to be downloaded manually.

### Load EIA facility data
Only need to keep the plant id, year (as a check that plants don't move between years), and lat/lon

In [3]:
path = os.path.join(data_path, 'Derived data',
                    'Facility gen fuels and CO2 {}.csv'.format(file_date))
facility_df = pd.read_csv(path)
facility_df['state'] = facility_df['geography'].str[-2:]

In [4]:
plants = facility_df.loc[:, ['plant id', 'year', 'lat', 'lon', 'state']]
plants.drop_duplicates(inplace=True)

### Load known NERC labels from EIA-860
Current NERCS go back to 2012. Use all annual 860 files from 2012 through the most recent available. Extend the dictionary of dictionaries below with any files available after 2016. `io`, `skiprows`, and `usecols` are all input parameters for the Pandas `read_excel` function.

In [5]:
eia_base_path = join(data_path, 'EIA downloads')
file_860_info = {
#     2011: {'io': join(eia_base_path, 'eia8602011', 'Plant.xlsx'),
#            'skiprows': 0,
#            'parse_cols': 'B,J'},
    2012: {'io': join(eia_base_path, 'eia8602012', 'PlantY2012.xlsx'),
           'skiprows': 0,
           'usecols': 'B,J'},
    2013: {'io': join(eia_base_path, 'eia8602013', '2___Plant_Y2013.xlsx'),
           'skiprows': 0,
           'usecols': 'C,L'},
    2014: {'io': join(eia_base_path, 'eia8602014', '2___Plant_Y2014.xlsx'),
           'skiprows': 0,
           'usecols': 'C,L'},
    2015: {'io': join(eia_base_path, 'eia8602015', '2___Plant_Y2015.xlsx'),
           'skiprows': 0,
           'usecols': 'C,L'},
    2016: {'io': join(eia_base_path, 'eia8602016', '2___Plant_Y2016.xlsx'),
           'skiprows': 0,
           'usecols': 'C,L'}
}

In [6]:
eia_nercs = {}
for key, args in file_860_info.items():
    eia_nercs[key] = pd.read_excel(**args)
    eia_nercs[key].columns = ['plant id', 'nerc']
    eia_nercs[key]['year'] = key

I want to assign NERC regions for every year. We have data for 2012 onward from the EIA-860 files. For the purpose of this analysis I'll assume that all years from 2001-2011 are the same NERC as 2012.

Also assume that values in 2017 are the same as in 2016. I'll fill in nerc values for plants that were built in 2017 below.

In [30]:
for year in range(2001, 2012):
    # the pandas .copy() method is deep by default but I'm not sure in this case
    df = deepcopy(eia_nercs[2012])
    df['year'] = year
    
    eia_nercs[year] = df
    
df = deepcopy(eia_nercs[2016])
df['year'] = 2017
eia_nercs[2017] = df

In [31]:
eia_nercs.keys()

dict_keys([2012, 2013, 2014, 2015, 2016, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2017])

In [32]:
eia_nercs[2001].head()

,plant id,nerc,year
0,10867,SERC,2001
1,50903,RFC,2001
2,10671,SPP,2001
3,2527,NPCC,2001
4,3305,SERC,2001


In [33]:
nercs = pd.concat(eia_nercs.values())
nercs.sort_values('year', inplace=True)

In [34]:
nercs.head()

,plant id,nerc,year
5465,56512,RFC,2001
4866,1481,NPCC,2001
4865,1480,NPCC,2001
4864,805,NPCC,2001
4863,54451,WECC,2001


In [13]:
nercs.year.unique()

array([2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016])

### Look for plants listed with different NERC labels

** This may not matter anymore if I use NERC labels for each year **

There are 30 plants duplicated. Five of them don't have a NERC label in one of the years. The largest move is from MRO to other regions (12), with most of those to SPP (7). After that, moves from RFC (5) to MRO (3) and SERC (2). There are also some moves from WECC and FRCC to HICC/ASCC - these might be diesel generators that get moved.

The plants that have duplicate NERC region labels represent a small fraction of national generation, but one that is growing over time. By 2016 they consist of 0.15% of national generation.

In [14]:
for df_ in list(eia_nercs.values()) + [nercs]:
    print('{} total records'.format(len(df_)))
    print('{} unique plants'.format(len(df_['plant id'].unique())))

7289 total records
7289 unique plants
8060 total records
8060 unique plants
8520 total records
8520 unique plants
8928 total records
8928 unique plants
9711 total records
9711 unique plants
7289 total records
7289 unique plants
7289 total records
7289 unique plants
7289 total records
7289 unique plants
7289 total records
7289 unique plants
7289 total records
7289 unique plants
7289 total records
7289 unique plants
7289 total records
7289 unique plants
7289 total records
7289 unique plants
7289 total records
7289 unique plants
7289 total records
7289 unique plants
7289 total records
7289 unique plants
122687 total records
10068 unique plants


In [9]:
dup_plants = nercs.loc[nercs['plant id'].duplicated(keep=False), 'plant id'].unique()
dup_plants

array([   66,  1120,  1121,  7757,  7848,  7847,  6280, 57251, 57252,
          70,   899,  1168, 57449, 58469, 55836, 56266, 56106, 56856,
       56985, 57622, 57623, 57651, 57650, 57983, 58117, 58278, 58511,
       59027, 59037, 58690, 58655, 58676])

In [10]:
region_list = []
for plant in dup_plants:
    regions = nercs.loc[nercs['plant id'] == plant, 'nerc'].unique()
#     regions = regions.tolist()
    region_list.append(regions)
Counter(tuple(x) for x in region_list)

Counter({('ASCC', nan): 2,
         ('FRCC', 'HICC'): 1,
         ('MRO', 'RFC'): 2,
         ('MRO', 'SERC'): 1,
         ('MRO', 'SPP'): 7,
         ('MRO', 'WECC'): 3,
         ('RFC', 'MRO'): 3,
         ('RFC', 'SERC'): 2,
         ('SERC', 'SPP'): 1,
         ('SPP', 'SERC'): 2,
         ('SPP', 'TRE'): 1,
         ('WECC', 'ASCC'): 2,
         ('WECC', 'HICC'): 1,
         (nan, 'WECC', 'ASCC'): 3,
         (nan, 'WECC', 'HICC'): 1})

In [11]:
(facility_df.loc[facility_df['plant id'].isin(dup_plants), :]
            .groupby('year')['generation (MWh)'].sum()
 / facility_df.loc[:, :]
              .groupby('year')['generation (MWh)'].sum())

year
2001    0.000345
2002    0.000269
2003    0.000262
2004    0.000313
2005    0.000426
2006    0.000514
2007    0.000509
2008    0.000527
2009    0.000631
2010    0.000683
2011    0.000763
2012    0.001286
2013    0.001138
2014    0.001079
2015    0.001701
2016    0.001962
2017    0.001460
Name: generation (MWh), dtype: float64

### Some plants in EIA-860 don't have NERC labels. Drop them now.
This is my training data. All of these plants should still be in my `plants` dataframe.

In [35]:
nan_plants = {}
all_nan = []
years = nercs.year.unique()
for year in years:
    nan_plants[year] = nercs.loc[(nercs.year == year) &
                                 (nercs.isnull().any(axis=1)), 'plant id'].tolist()
    all_nan.extend(nan_plants[year])

# number of plants that don't have a nerc in at least one year
len(all_nan)

232

In [37]:
# drop all the rows without a nerc value
nercs.dropna(inplace=True)

In [38]:
nan_plants[2017]

[58651,
 58656,
 58659,
 58662,
 58639,
 58640,
 58549,
 58684,
 58277,
 58380,
 58425,
 58405,
 60563,
 60588,
 60260,
 60250,
 60243,
 60244,
 60245,
 60328,
 61166,
 61172,
 61364,
 60814,
 61099,
 61101,
 61068,
 58989,
 58982,
 58977,
 58837,
 59035,
 60125,
 60024,
 66,
 70]

## Load EIA-860m for some info on recent facilities
The EIA-860m (monthly) data file has an up-to-date list of all operating power plants and their associated balancing authority. It does not list the NERC region, so it can't be used to assign NERC labels for all plants. But in some cases knowing the state and balancing authority is enough to make a good guess about which NERC a plant is in.

Assigning NERC region labels has the lowest accuracy for plants in SPP and TRE. To compensate, I'm going to assume that anything in TX or OK and SWPP balancing authority is in SPP. On the flip side, if it's in TX and ERCOT I'll assign it to TRE.

Only do this for plants that come online since the most recent 860 annual data.

In [40]:
path = join(data_path, 'EIA downloads', 'december_generator2017.xlsx')

# Check the excel file columns if there is a read error. They should match
# the plant id, plant state, operating year, and balancing authority code.
m860 = pd.read_excel(path, sheet_name='Operating',skip_footer=1,
                     usecols='C,F,P,AE', skiprows=0)
m860.columns = m860.columns.str.lower()

In [41]:
# most_recent_860_year is defined at the top of this notebook
# The goal here is to only look at plants that started operating after
# the most recent annual data

m860 = m860.loc[m860['operating year'] > most_recent_860_year]
m860.tail()

,plant id,plant state,operating year,balancing authority code
21373,61632,IA,2017,MISO
21374,61633,MA,2017,ISNE
21375,61634,MA,2017,ISNE
21376,61635,MA,2017,ISNE
21377,61636,MA,2017,ISNE


In [42]:
m860.loc[(m860['plant state'].isin(['TX', 'OK'])) &
         (m860['balancing authority code'] == 'SWPP'), 'nerc'] = 'SPP'

m860.loc[(m860['plant state'].isin(['TX'])) &
         (m860['balancing authority code'] == 'ERCO'), 'nerc'] = 'TRE'

In [43]:
# Drop all rows except the ones I've labeled as TRE or SPP
m860.dropna(inplace=True)
m860.head()

,plant id,plant state,operating year,balancing authority code,nerc
343,165,OK,2017,SWPP,SPP
344,165,OK,2017,SWPP,SPP
4836,2953,OK,2017,SWPP,SPP
4837,2953,OK,2017,SWPP,SPP
4838,2953,OK,2017,SWPP,SPP


Make lists of plant codes for SPP and TRE facilities

In [29]:
nercs.head()

,plant id,nerc,year
5465,56512,RFC,2001
4866,1481,NPCC,2001
4865,1480,NPCC,2001
4864,805,NPCC,2001
4863,54451,WECC,2001


In [55]:
# Create additional dataframes with 2017 SPP and TRE plants.
# Use these to fill in values for 2017 plants

m860_spp_plants = (m860.loc[m860['nerc'] == 'SPP', 'plant id']
                       .drop_duplicates()
                       .reset_index(drop=True))

additional_spp = pd.DataFrame(m860_spp_plants.copy())
# additional_spp['plant id'] = m860_spp_plants
additional_spp['nerc'] = 'SPP'
additional_spp['year'] = 2017

m860_tre_plants = (m860.loc[m860['nerc'] == 'TRE', 'plant id']
                       .drop_duplicates()
                       .reset_index(drop=True))

additional_tre = pd.DataFrame(m860_tre_plants)
# additional_tre['plant id'] = m860_tre_plants
additional_tre['nerc'] = 'TRE'
additional_tre['year'] = 2017

In [57]:
additional_spp

,plant id,nerc,year
0,165,SPP,2017
1,2953,SPP,2017
2,60414,SPP,2017
3,61221,SPP,2017
4,61261,SPP,2017
5,61614,SPP,2017
6,61615,SPP,2017
7,61616,SPP,2017
8,61617,SPP,2017
9,61618,SPP,2017


In [58]:
additional_tre

,plant id,nerc,year
0,56984,TRE,2017
1,59066,TRE,2017
2,59193,TRE,2017
3,59206,TRE,2017
4,59245,TRE,2017
5,59712,TRE,2017
6,59812,TRE,2017
7,60122,TRE,2017
8,60210,TRE,2017
9,60217,TRE,2017


### Append my 2017 SPP and TRE guesses to the full nerc dataframe

In [60]:
nercs = pd.concat([nercs, additional_spp, additional_tre])

## Clean and prep data for KNN

In [61]:
plants.head()

,plant id,year,lat,lon,state
0,1001,2017,39.9242,-87.4244,IN
12,1001,2016,39.9242,-87.4244,IN
24,1001,2015,39.9242,-87.4244,IN
36,1001,2014,39.9242,-87.4244,IN
48,1001,2013,39.9242,-87.4244,IN


In [65]:
plants.loc[]

,plant id,year,lat,lon,state
1647344,1275,2006,38.947222,99.528611,KS
1647356,1275,2005,38.947222,99.528611,KS
1647368,1275,2004,38.947222,99.528611,KS
1647380,1275,2003,38.947222,99.528611,KS
1647392,1275,2002,38.947222,99.528611,KS


In [62]:
nercs.tail()

,plant id,nerc,year
24,61309,TRE,2017
25,61362,TRE,2017
26,61409,TRE,2017
27,61410,TRE,2017
28,61411,TRE,2017


Checked to make sure the type of merge doesn't matter once rows without nerc values are dropped

In [63]:
df = pd.merge(plants, nercs, on=['plant id', 'year'], how='left')

In [78]:
df_inner = pd.merge(plants, nercs, on=['plant id', 'year'], how='inner')

In [90]:
pd.testing.assert_frame_equal(df_inner.dropna(subset=['nerc']).reset_index(drop=True),
                              df.dropna(subset=['nerc']).reset_index(drop=True))

In [75]:
omitted = set(df['plant id'].unique()) - set(nercs['plant id'].unique())

In [64]:
df.head()

,plant id,year,lat,lon,state,nerc
0,1001,2017,39.9242,-87.4244,IN,RFC
1,1001,2016,39.9242,-87.4244,IN,RFC
2,1001,2015,39.9242,-87.4244,IN,RFC
3,1001,2014,39.9242,-87.4244,IN,RFC
4,1001,2013,39.9242,-87.4244,IN,RFC


In [66]:
df.tail()

,plant id,year,lat,lon,state,nerc
94688,1275,2006,38.947222,99.528611,KS,SPP
94689,1275,2005,38.947222,99.528611,KS,SPP
94690,1275,2004,38.947222,99.528611,KS,SPP
94691,1275,2003,38.947222,99.528611,KS,SPP
94692,1275,2002,38.947222,99.528611,KS,SPP


In [52]:
df.columns

Index(['plant id', 'year', 'lat', 'lon', 'state', 'nerc'], dtype='object')

Drop plants that don't have lat/lon data (using just lon to check), and then drop duplicates. If any plants have kept the same plant id but moved over time (maybe a diesel generator?) or switched NERC they will show up twice.

In [94]:
df.loc[df.lon.isnull()].drop_duplicates(subset='plant id')

,plant id,year,lat,lon,state,nerc
86689,10851,2006,NaN,NaN,NJ,NaN
86877,50291,2005,NaN,NaN,FL,NaN
87396,56672,2010,NaN,NaN,MN,NaN
87597,55982,2004,NaN,NaN,AK,NaN
87646,55150,2004,NaN,NaN,LA,NaN
87679,55082,2006,NaN,NaN,MS,NaN
87744,55521,2005,NaN,NaN,CA,NaN
87810,55314,2003,NaN,NaN,NaN,NaN
88107,54243,2005,NaN,NaN,GA,NaN
88230,50726,2007,NaN,NaN,LA,NaN


In [96]:
cols = ['plant id', 'lat', 'lon', 'nerc', 'state', 'year']
df_slim = (df.loc[:, cols].dropna(subset=['lon'])
             .drop_duplicates(subset=['plant id', 'year', 'nerc']))

In [97]:
len(df_slim)

94525

In [98]:
df_slim.head()

,plant id,lat,lon,nerc,state,year
0,1001,39.9242,-87.4244,RFC,IN,2017
1,1001,39.9242,-87.4244,RFC,IN,2016
2,1001,39.9242,-87.4244,RFC,IN,2015
3,1001,39.9242,-87.4244,RFC,IN,2014
4,1001,39.9242,-87.4244,RFC,IN,2013


Separate out the list of plants where we don't have NERC labels from EIA-860.

In [100]:
unknown = df_slim.loc[df_slim.nerc.isnull()].copy()

In [102]:
print("{} plants/years don't have NERC labels\n".format(len(unknown)))
print(unknown.head())

1552 plants/years don't have NERC labels

       plant id    lat     lon nerc state  year
28656      3823  38.27 -78.035  NaN    VA  2009
28657      3823  38.27 -78.035  NaN    VA  2005
28658      3823  38.27 -78.035  NaN    VA  2004
28659      3823  38.27 -78.035  NaN    VA  2003
28660      3823  38.27 -78.035  NaN    VA  2002


### Create X and y matricies
X is lat/lon and year

y is the NERC label

For both, I'm only using plants where we have all data (no `NaN`s). Not doing any transformation of the lat/lon at this time.

In [120]:
X = df_slim.loc[df_slim.notnull().all(axis=1), ['lat', 'lon', 'year']]
y = df_slim.loc[df_slim.notnull().all(axis=1), 'nerc']

In [121]:
len(X)

92973

In [134]:
# Make sure that unknown and X include all records from df_slim
len(X) + len(unknown) - len(df_slim)

0

In [122]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

## GridSearch to find the best parameters

In [123]:
from sklearn.ensemble import RandomForestClassifier


In [243]:
rf = RandomForestClassifier()
params = dict(
    n_estimators = [5, 10, 25, 50],
    min_samples_split = [2, 5, 10],
    min_samples_leaf = [1, 3, 5],
)

clf_rf = GridSearchCV(rf, params, n_jobs=-1, iid=False, verbose=1)
clf_rf.fit(X_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   25.8s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:  1.1min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=False, n_jobs=-1,
       param_grid={'n_estimators': [5, 10, 25, 50], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 3, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [125]:
clf_rf.best_estimator_, clf_rf.best_score_

(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False), 0.9835771963992802)

In [126]:
clf_rf.score(X_test, y_test)

0.982595658692393

In [127]:
nerc_labels = nercs.nerc.dropna().unique()

Accuracy score by region

In [128]:
for region in nerc_labels:
    mask = y_test == region
    
    X_masked = X_test[mask]
    y_hat_masked = clf_rf.predict(X_masked)
    y_test_masked = y_test[mask]
    
    accuracy = metrics.accuracy_score(y_test_masked, y_hat_masked)
    print('{} : {}'.format(region, accuracy))

RFC : 0.9707534379236877
NPCC : 0.9964832956543582
WECC : 0.9983339283589194
SERC : 0.9778310365488316
SPP : 0.9159561510353228
TRE : 0.9803773584905661
MRO : 0.9809056192034915
FRCC : 0.9860917941585535
ASCC : 1.0
HICC : 1.0


F1 score by region

In [129]:
y_hat = clf_rf.predict(X_test)

for region in nerc_labels:
    f1 = metrics.f1_score(y_test, y_hat, labels=[region], average='macro')
    print('{} : {}'.format(region, f1))

RFC : 0.9713178294573643
NPCC : 0.9954830614805521
WECC : 0.9982151356496907
SERC : 0.9729729729729729
SPP : 0.9382407985028073
TRE : 0.9852104664391353
MRO : 0.9753186872796311
FRCC : 0.9895324494068388
ASCC : 1.0
HICC : 1.0


## Plants without lat/lon

In [220]:
cols = ['plant id', 'nerc', 'state', 'year']
df_state_slim = (df.loc[:, cols].dropna(subset=['state']).copy())

In [221]:
df_state_slim.head()

,plant id,nerc,state,year
0,1001,RFC,IN,2017
1,1001,RFC,IN,2016
2,1001,RFC,IN,2015
3,1001,RFC,IN,2014
4,1001,RFC,IN,2013


In [222]:
len(df_state_slim)

94668

In [223]:
le = LabelEncoder()

In [224]:
df_state_slim.loc[:, 'enc state'] = le.fit_transform(df_state_slim.loc[:, 'state'].tolist())

In [225]:
unknown_state = df_state_slim.loc[df_state_slim.nerc.isnull()].copy()

In [197]:
df_state_slim.dtypes

plant id       int64
year           int64
lat          float64
lon          float64
state         object
nerc          object
enc state      int64
dtype: object

In [195]:
le.inverse_transform(df_state_slim.loc[:, 'enc state'])

/Users/Home/anaconda/envs/psci/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['IN', 'IN', 'IN', ..., 'KS', 'KS', 'KS'], dtype='<U2')

In [226]:
X_state = df_state_slim.loc[df_state_slim.notnull().all(axis=1), ['enc state', 'year']].copy()
y_state = df_state_slim.loc[df_state_slim.notnull().all(axis=1), 'nerc'].copy()

In [227]:
y_state.isnull().any()

False

In [228]:
X_state_train, X_state_test, y_state_train, y_state_test = train_test_split(
    X_state, y_state, test_size=0.33, random_state=42)

In [233]:
rf = RandomForestClassifier()
params = dict(
    n_estimators = [5, 10, 25, 50],
    min_samples_split = [2, 5, 10],
    min_samples_leaf = [1, 3, 5],
)

clf_rf_state = GridSearchCV(rf, params, n_jobs=-1, iid=False, verbose=1)
clf_rf_state.fit(X_state_train, y_state_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   15.3s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:   42.7s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=False, n_jobs=-1,
       param_grid={'n_estimators': [5, 10, 25, 50], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 3, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [235]:
clf_rf_state.best_estimator_, clf_rf_state.best_score_

(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=5,
             min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False), 0.9393516437547577)

In [236]:
clf_rf_state.score(X_state_test, y_state_test)

0.9382209188660802

In [237]:
nerc_labels = nercs.nerc.dropna().unique()

Accuracy score by region

In [238]:
for region in nerc_labels:
    mask = y_state_test == region
    
    X_state_masked = X_state_test[mask]
    y_state_hat_masked = clf_rf_state.predict(X_state_masked)
    y_state_test_masked = y_state_test[mask]
    
    accuracy = metrics.accuracy_score(y_state_test_masked, y_state_hat_masked)
    print('{} : {}'.format(region, accuracy))

RFC : 0.9220880405142189
NPCC : 0.9957211175434181
WECC : 0.9937426210153483
SERC : 0.8845998383185125
SPP : 0.5650869826034793
TRE : 1.0
MRO : 0.9643738010413812
FRCC : 1.0
ASCC : 1.0
HICC : 1.0


F1 score by region

In [240]:
y_state_hat = clf_rf_state.predict(X_state_test)

for region in nerc_labels:
    f1 = metrics.f1_score(y_state_test, y_state_hat, labels=[region], average='macro')
    print('{} : {}'.format(region, f1))

RFC : 0.9102095750817151
NPCC : 0.9965990678926816
WECC : 0.992629282386933
SERC : 0.879003916055829
SPP : 0.7196333078686019
TRE : 0.8811881188118811
MRO : 0.9515954570037858
FRCC : 0.9669876203576341
ASCC : 0.9991673605328892
HICC : 1.0


### Regular KNN classifier
Run gridsearch testing parameter values for weights, n_neighbors, and p (use Euclidean or Manhattan distance).

With 15 neighbors, weights by distance, and Euclidean distance, the model is able to accurately predict the test sample NERC region with 96% accuracy. This varies by region, with the lowest accuracy scores for TRE and SPP (89% and 87%), and the highest accuracy scores for WECC and NPCC (each 99%). F1 scores tend to be similar to the accuracy, although TRE has slightly higher F1 (0.94 vs 0.89).

In [55]:
knn = neighbors.KNeighborsClassifier()

params = {'weights': ['uniform', 'distance'],
          'n_neighbors': [10, 15, 20],
          'p': [1, 2]
         }

clf_knn = GridSearchCV(knn, params, n_jobs=-1, iid=False, verbose=1)

clf_knn.fit(X_train, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    0.7s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=False, n_jobs=-1,
       param_grid={'weights': ['uniform', 'distance'], 'n_neighbors': [10, 15, 20], 'p': [1, 2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [35]:
clf_knn.best_estimator_, clf_knn.best_score_

(KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
            metric_params=None, n_jobs=1, n_neighbors=10, p=1,
            weights='distance'), 0.9590098375679993)

In [36]:
clf_knn.score(X_test, y_test)

0.9589136490250696

In [37]:
nerc_labels = nercs.nerc.dropna().unique()

Accuracy score by region

In [38]:
for region in nerc_labels:
    mask = y_test == region
    
    X_masked = X_test[mask]
    y_hat_masked = clf_knn.predict(X_masked)
    y_test_masked = y_test[mask]
    
    accuracy = metrics.accuracy_score(y_test_masked, y_hat_masked)
    print('{} : {}'.format(region, accuracy))

SERC : 0.9553752535496958
RFC : 0.9523809523809523
SPP : 0.8098591549295775
NPCC : 0.9768115942028985
WECC : 0.9904191616766467
MRO : 0.9469964664310954
TRE : 0.927007299270073
HICC : 1.0
ASCC : 0.975609756097561
FRCC : 0.9629629629629629


F1 score by region

In [39]:
y_hat = clf_knn.predict(X_test)

for region in nerc_labels:
    f1 = metrics.f1_score(y_test, y_hat, labels=[region], average='macro')
    print('{} : {}'.format(region, f1))

SERC : 0.9534412955465588
RFC : 0.949667616334283
SPP : 0.8273381294964028
NPCC : 0.9810771470160117
WECC : 0.9904191616766467
MRO : 0.9337979094076655
TRE : 0.9407407407407407
HICC : 0.9444444444444444
ASCC : 0.975609756097561
FRCC : 0.9811320754716981


In [40]:
metrics.f1_score(y_test, y_hat, average='micro')

0.9589136490250696

In [41]:
metrics.f1_score(y_test, y_hat, average='macro')

0.9477668276232013

## Use best RandomForest parameters to predict NERC for unknown plants

In [244]:
unknown.loc[:, 'nerc'] = clf_rf.predict(unknown.loc[:, ['lat', 'lon', 'year']])
unknown_state.loc[:, 'nerc'] = clf_rf_state.predict(unknown_state.loc[:, ['enc state', 'year']])

Ensuring that no plants in Alaska or Hawaii are assigned to continental NERCs, or the other way around.

In [131]:
print(unknown.loc[unknown.state.isin(['AK', 'HI']), 'nerc'].unique())
print(unknown.loc[unknown.nerc.isin(['HICC', 'ASCC']), 'state'].unique())

['ASCC' 'HICC']
['AK' 'HI']


In [135]:
Counter(unknown['nerc'])

Counter({'ASCC': 39,
         'FRCC': 5,
         'HICC': 34,
         'MRO': 74,
         'NPCC': 251,
         'RFC': 244,
         'SERC': 301,
         'SPP': 96,
         'TRE': 153,
         'WECC': 355})

In [137]:
unknown.head()

,plant id,lat,lon,nerc,state,year
28656,3823,38.27,-78.035,SERC,VA,2009
28657,3823,38.27,-78.035,SERC,VA,2005
28658,3823,38.27,-78.035,SERC,VA,2004
28659,3823,38.27,-78.035,SERC,VA,2003
28660,3823,38.27,-78.035,SERC,VA,2002


## Export plants with lat/lon, state, and nerc

In [138]:
nercs.tail()

,plant id,nerc,year
24,61309,TRE,2017
25,61362,TRE,2017
26,61409,TRE,2017
27,61410,TRE,2017
28,61411,TRE,2017


In [139]:
unknown.head()

,plant id,lat,lon,nerc,state,year
28656,3823,38.27,-78.035,SERC,VA,2009
28657,3823,38.27,-78.035,SERC,VA,2005
28658,3823,38.27,-78.035,SERC,VA,2004
28659,3823,38.27,-78.035,SERC,VA,2003
28660,3823,38.27,-78.035,SERC,VA,2002


In [251]:
unknown_state.tail()

,plant id,nerc,state,year,enc state
94592,52205,WECC,CA,2002,4
94593,52205,WECC,CA,2001,4
94604,56508,WECC,CA,2007,4
94684,596,RFC,DE,2004,8
94685,596,RFC,DE,2003,8


In [141]:
df_slim.head()

,plant id,lat,lon,nerc,state,year
0,1001,39.9242,-87.4244,RFC,IN,2017
1,1001,39.9242,-87.4244,RFC,IN,2016
2,1001,39.9242,-87.4244,RFC,IN,2015
3,1001,39.9242,-87.4244,RFC,IN,2014
4,1001,39.9242,-87.4244,RFC,IN,2013


In [252]:
labeled = pd.concat([df_slim.loc[df_slim.notnull().all(axis=1)],
                     unknown,
                     unknown_state.loc[:, ['plant id', 'nerc', 'state', 'year']]])

In [254]:
labeled.tail()

,lat,lon,nerc,plant id,state,year
94592,NaN,NaN,WECC,52205,CA,2002
94593,NaN,NaN,WECC,52205,CA,2001
94604,NaN,NaN,WECC,56508,CA,2007
94684,NaN,NaN,RFC,596,DE,2004
94685,NaN,NaN,RFC,596,DE,2003


In [253]:
labeled.loc[labeled.nerc.isnull()]

,lat,lon,nerc,plant id,state,year


There are 7 facilities that don't show up in my labeled data.

In [257]:
facility_df.loc[~facility_df['plant id'].isin(labeled['plant id']), 'plant id'].unique()

array([55314, 55952,  7704,  6339, 55975, 55073, 50728])

In [258]:
path = join(data_path, 'Facility labels', 'Facility locations_knn.csv')
labeled.to_csv(path, index=False)